Aqui está uma sugestão de descrição em Markdown para o seu novo notebook. Você pode colar isso na primeira célula (como texto) ou no `README.md` do projeto.

---

# 📊 Validação Estatística de Modelos via Engenharia Reversa de Métricas

## 🎯 Objetivo

Este notebook tem como finalidade realizar **testes de significância estatística (Teste T)** comparando o desempenho de diferentes modelos de classificação (ex: Llama Zero-Shot vs. Few-Shot, BERT, RoBERTa), utilizando **apenas** os dados agregados (Média e Intervalo de Confiança) disponíveis no CSV de resultados finais, sem a necessidade de reprocessar os dados brutos de cada fold.

## 🛠️ Utilidade e Funcionamento

Este código resolve o problema de comparar se a diferença entre dois modelos é **estatisticamente relevante** ou obra do acaso, mesmo quando não temos acesso imediato aos vetores de predição originais.

### 1. Engenharia Reversa do Intervalo de Confiança (IC)

Como os resultados originais foram gerados utilizando a distribuição **T-Student** com 5 folds, este notebook consegue recuperar o **Erro Padrão ()** original através da fórmula:

Onde a *Margem* é extraída da string do CSV (ex: a diferença entre a média `0.594` e o limite superior `0.607` em "0.594 (0.581, 0.607)").

### 2. Teste T de Welch (Welch's t-test)

Com as médias e os erros padrão recuperados, o notebook aplica o **Teste T de Welch**. Esta é a variação mais robusta do teste T para duas amostras independentes, pois:

* Não assume que as variâncias dos dois modelos são iguais.
* Calcula os graus de liberdade de forma ajustada (equação de Welch-Satterthwaite).

### 3. Tomada de Decisão

O notebook retorna um **P-Valor** (-value).

* Se : Rejeita-se a hipótese nula. Conclui-se que os modelos são **diferentes** (um é de fato superior ao outro).
* Se : Aceita-se a hipótese nula. A diferença numérica observada pode ser apenas ruído estatístico; os modelos são estatisticamente **equivalentes**.

## 🚀 Como Usar

A função `calcular_teste_t_do_csv` exige apenas:

1. O **DataFrame** carregado do CSV.
2. O **nome exato** dos dois modelos a serem comparados (coluna `model/test`).
3. A **métrica** alvo (ex: `macro_f1`, `class1f1`).

---



In [9]:
import pandas as pd
import numpy as np
import re
from scipy import stats

def extrair_estatisticas(texto_celula):
    """
    Transforma string '0.594 (0.581, 0.607)' em média, limite_inferior, limite_superior.
    """
    try:
        # Regex para capturar números decimais no formato X.XXX (Y.YYY, Z.ZZZ)
        match = re.search(r"(\d+\.?\d*)\s*\(\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*\)", str(texto_celula))
        if match:
            media = float(match.group(1))
            inferior = float(match.group(2))
            superior = float(match.group(3))
            return media, inferior, superior
    except Exception as e:
        return None, None, None
    return None, None, None

In [10]:
def calcular_teste_t_do_csv(df, modelo_a, modelo_b, coluna_metrica, n_folds=5, alpha=0.05):
    """
    Realiza o Teste T de Welch comparando dois modelos baseados apenas na string de média e IC.

    Parâmetros:
      df: DataFrame carregado do CSV.
      modelo_a: String exata do nome do primeiro modelo (coluna 'model/test').
      modelo_b: String exata do nome do segundo modelo.
      coluna_metrica: Nome da coluna (ex: 'macro_f1', 'class1f1').
      n_folds: Tamanho da amostra usado para gerar o intervalo (padrão 5 para k-fold).
      alpha: Nível de significância (padrão 0.05 para 95% de confiança).
    """

    # 1. Obter as linhas dos modelos
    linha_a = df[df['model/test'] == modelo_a]
    linha_b = df[df['model/test'] == modelo_b]

    if linha_a.empty or linha_b.empty:
        return f"Erro: Um dos modelos não foi encontrado no DataFrame.\nBusca A: {modelo_a}\nBusca B: {modelo_b}"

    val_a = linha_a[coluna_metrica].values[0]
    val_b = linha_b[coluna_metrica].values[0]

    # 2. Extrair dados numéricos da string
    media_a, inf_a, sup_a = extrair_estatisticas(val_a)
    media_b, inf_b, sup_b = extrair_estatisticas(val_b)

    if media_a is None or media_b is None:
        return "Erro: Falha ao fazer o parsing dos valores na coluna selecionada."

    # 3. Engenharia Reversa do Erro Padrão (Standard Error - SE)
    # Fórmula do IC: Media +/- (t_critico * SE)
    # Logo: SE = (Superior - Inferior) / (2 * t_critico)

    # Graus de liberdade (df) para obter o t_critico usado na geração do CSV
    df_graus_liberdade = n_folds - 1

    # Valor crítico T usado originalmenta para 95% de confiança (bicaudal)
    t_critico_original = stats.t.ppf(0.975, df_graus_liberdade)

    # Margem de erro (metade do tamanho do intervalo)
    margem_erro_a = (sup_a - inf_a) / 2
    margem_erro_b = (sup_b - inf_b) / 2

    # Recalculando o Erro Padrão (SE)
    se_a = margem_erro_a / t_critico_original
    se_b = margem_erro_b / t_critico_original

    # 4. Executar Teste T de Welch (para amostras independentes com variâncias possivelmente diferentes)
    # Estatística T = (Media1 - Media2) / sqrt(SE1^2 + SE2^2)
    numerador = media_a - media_b
    denominador = np.sqrt(se_a**2 + se_b**2)

    if denominador == 0:
        return "As médias e intervalos são idênticos (t = 0)."

    t_stat = numerador / denominador

    # Graus de liberdade aproximados (Welch-Satterthwaite equation)
    dof_welch = ((se_a**2 + se_b**2)**2) / (((se_a**2)**2 / (n_folds-1)) + ((se_b**2)**2 / (n_folds-1)))

    # P-Valor (bicaudal)
    p_valor = stats.t.sf(np.abs(t_stat), dof_welch) * 2

    # 5. Interpretação
    significativo = p_valor < alpha
    resultado = "DIFERENTE (Estatísticamente Significativo)" if significativo else "IGUAL (Não há evidência estatística de diferença)"

    relatorio = {
        "Modelo A": modelo_a,
        "Média A": media_a,
        "Modelo B": modelo_b,
        "Média B": media_b,
        "Diferença": media_a - media_b,
        "P-Valor": p_valor,
        "T-Statistic": t_stat,
        "Conclusão": resultado
    }

    return relatorio


In [11]:
from google.colab import drive

drive.mount('/content/drive')
caminho = '/content/drive/MyDrive/red_pill_analises/classificador/testes_estatisticos/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Carregando os dataframes e limpando os nomes dos modelos para garantir que os testes funcionem

In [12]:
# Carregando os dataframes
df_misoginia_videos = pd.read_csv(caminho + 'resultados_modelos_unificados_misoginia_videos.csv')
df_toxic_videos = pd.read_csv(caminho + 'resultados_modelos_unificados_toxic_videos.csv')
df_misoginia_comentarios = pd.read_csv(caminho + 'resultados_modelos_unificados_misoginia_comentarios.csv')
df_toxic_comentarios = pd.read_csv(caminho + 'resultados_modelos_unificados_toxic_comentarios.csv')

In [13]:
import pandas as pd
import re

# Lista dos seus dataframes
dfs = [df_misoginia_videos, df_toxic_videos, df_misoginia_comentarios, df_toxic_comentarios]

# Regex para identificar 'misoginia' ou 'toxicidade' (case insensitive)
# \s* garante que remova espaços colados na palavra também
padrao_remocao = r'\s*(misoginia|toxicidade)\s*'

for df in dfs:
    if 'model/test' in df.columns:
        # 1. Remove as palavras substituindo por um espaço único
        df['model/test'] = df['model/test'].str.replace(
            padrao_remocao,
            ' ',
            flags=re.IGNORECASE,
            regex=True
        )

        # 2. Limpeza fina: remove espaços duplos que sobraram e espaços nas pontas
        df['model/test'] = df['model/test'].str.replace(r'\s+', ' ', regex=True).str.strip()

print("Limpeza concluída!")

# Verificando como ficou em um deles (exemplo)
print(df_misoginia_videos['model/test'].head())

Limpeza concluída!
0    LLama Zero-Shot (Teste 1). portugues sem conceito
1        LLama Zero-Shot (Teste 2) ingles sem conceito
2    LLama Zero-Shot (Teste 3), portugues com conceito
3       LLama Zero-Shot (Teste 4), ingles com conceito
4                             LLama Few-Shot (Teste 1)
Name: model/test, dtype: object


In [14]:
import pandas as pd

# Supondo que você já carregou os dataframes como no seu exemplo:
# df_misoginia_videos = ...
# df_toxic_videos = ...
# df_misoginia_comentarios = ...
# df_toxic_comentarios = ...

# Dicionário para facilitar a identificação no print
dfs = {
    "df_misoginia_videos": df_misoginia_videos,
    "df_toxic_videos": df_toxic_videos,
    "df_misoginia_comentarios": df_misoginia_comentarios,
    "df_toxic_comentarios": df_toxic_comentarios
}

# Usar o primeiro como referência
ref_name = "df_misoginia_videos"
ref_df = dfs[ref_name]
coluna = "model/test"

print(f"--- Iniciando verificação de consistência na coluna '{coluna}' ---\n")

tudo_igual = True

for name, df in dfs.items():
    if name == ref_name:
        continue

    # 1. Verifica se o tamanho é o mesmo
    if len(df) != len(ref_df):
        print(f"ERRO: {name} tem tamanho diferente ({len(df)}) de {ref_name} ({len(ref_df)})!")
        tudo_igual = False
        continue

    # 2. Compara a coluna valor a valor
    # Reset index garante que estamos comparando linha a linha pela posição, ignorando índices numéricos bagunçados
    comparacao = df[coluna].reset_index(drop=True) == ref_df[coluna].reset_index(drop=True)

    if not comparacao.all():
        tudo_igual = False
        print(f"DIFERENÇA ENCONTRADA em: {name}")

        # Pega os índices onde há diferença
        indices_diferentes = comparacao[~comparacao].index

        for idx in indices_diferentes:
            val_ref = ref_df[coluna].iloc[idx]
            val_df = df[coluna].iloc[idx]
            print(f"   Linha {idx}:")
            print(f"     {ref_name}: {val_ref}")
            print(f"     {name}: {val_df}")
        print("-" * 30)

if tudo_igual:
    print("✅ Sucesso! Todas as linhas da coluna 'model/test' são idênticas e estão na mesma ordem em todos os arquivos.")
else:
    print("\n❌ Foram encontradas inconsistências (veja acima).")

--- Iniciando verificação de consistência na coluna 'model/test' ---

✅ Sucesso! Todas as linhas da coluna 'model/test' são idênticas e estão na mesma ordem em todos os arquivos.


# Carregando os nomes dos modelos

In [15]:
llama_zero_shot_pt_sem_conceito = 'LLama Zero-Shot (Teste 1). portugues sem conceito'
llama_zero_shot_en_sem_conceito = 'LLama Zero-Shot (Teste 2) ingles sem conceito'
llama_zero_shot_pt_com_conceito = 'LLama Zero-Shot (Teste 3), portugues com conceito'
llama_zero_shot_en_com_conceito = 'LLama Zero-Shot (Teste 4), ingles com conceito'

llama_few_shot_pt_sem_conceito = 'LLama Few-Shot (Teste 1)'
llama_few_shot_en_sem_conceito = 'LLama Few-Shot (Teste 2)'
llama_few_shot_pt_com_conceito = 'LLama Few-Shot (Teste 3)'
llama_few_shot_en_com_conceito = 'LLama Few-Shot (Teste 4)'

llama_few_shot_pt_sem_conceito_roberta = 'LLama Few-Shot Embeddings RoBERTa (Teste 1)'
llama_few_shot_en_sem_conceito_roberta = 'LLama Few-Shot Embeddings RoBERTa (Teste 2)'
llama_few_shot_pt_com_conceito_roberta = 'LLama Few-Shot Embeddings RoBERTa (Teste 3)'
llama_few_shot_en_com_conceito_roberta = 'LLama Few-Shot Embeddings RoBERTa (Teste 4)'

llama_few_shot_pt_sem_conceito_bertimbau = 'LLama Few-Shot Embeddings BERTimbau (Teste 1)'
llama_few_shot_en_sem_conceito_bertimbau = 'LLama Few-Shot Embeddings BERTimbau (Teste 2)'
llama_few_shot_pt_com_conceito_bertimbau = 'LLama Few-Shot Embeddings BERTimbau (Teste 3)'
llama_few_shot_en_com_conceito_bertimbau = 'LLama Few-Shot Embeddings BERTimbau (Teste 4)'

llama_few_shot_pt_sem_conceito_minilm = 'LLama Few-Shot Embeddings MiniLM (Teste 1)'
llama_few_shot_en_sem_conceito_minilm = 'LLama Few-Shot Embeddings MiniLM (Teste 2)'
llama_few_shot_pt_com_conceito_minilm = 'LLama Few-Shot Embeddings MiniLM (Teste 3)'
llama_few_shot_en_com_conceito_minilm = 'LLama Few-Shot Embeddings MiniLM (Teste 4)'

llama_few_shot_pt_sem_conceito_qwen3_06b = 'LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 1)'
llama_few_shot_en_sem_conceito_qwen3_06b = 'LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 2)'
llama_few_shot_pt_com_conceito_qwen3_06b = 'LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 3)'
llama_few_shot_en_com_conceito_qwen3_06b = 'LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 4)'

llama_few_shot_pt_sem_conceito_qwen3_4b = 'LLama Few-Shot Embeddings Qwen3 - 4B (Teste 1)'
llama_few_shot_en_sem_conceito_qwen3_4b = 'LLama Few-Shot Embeddings Qwen3 - 4B (Teste 2)'
llama_few_shot_pt_com_conceito_qwen3_4b = 'LLama Few-Shot Embeddings Qwen3 - 4B (Teste 3)'
llama_few_shot_en_com_conceito_qwen3_4b = 'LLama Few-Shot Embeddings Qwen3 - 4B (Teste 4)'

llama_few_shot_pt_sem_conceito_qwen3_8b = 'LLama Few-Shot Embeddings Qwen3 - 8B (Teste 1)'
llama_few_shot_en_sem_conceito_qwen3_8b = 'LLama Few-Shot Embeddings Qwen3 - 8B (Teste 2)'
llama_few_shot_pt_com_conceito_qwen3_8b = 'LLama Few-Shot Embeddings Qwen3 - 8B (Teste 3)'
llama_few_shot_en_com_conceito_qwen3_8b = 'LLama Few-Shot Embeddings Qwen3 - 8B (Teste 4)'

bertimbau = 'BerTimbau'
roberta_desbalanceado = 'RoBERTa - - desbalanceado'
roberta_balanceado_50_50 = 'RoBERTa - - balanceado 50/50'
roberta_balanceado_60_40 = 'RoBERTa - - balanceado 60/40'

reg_log_tf_idf = 'Regressao Logistica TF-IDF (sem stop words)'
reg_log_tf_idf_stop_words = 'Regressao Logistica TF-IDF (stop words padrão)'
reg_log_tf_idf_stop_words_customizadas = 'Regressao Logistica TF-IDF (stop words customizadas)'
reg_log_tf_idf_stemming = 'Regressao Logistica TF-IDF (sem stop words + stemming)'
reg_log_tf_idf_stop_words_stemming = 'Regressao Logistica TF-IDF (stop words padrão + stemming)'
reg_log_tf_idf_stop_words_customizadas_stemming = 'Regressao Logistica TF-IDF (stop words customizadas + stemming)'

reg_log_roberta = 'Regressão Logística com entrada os embeddings do Roberta'
reg_log_roberta_ajuste = 'Regressão Logística com entrada os embeddings do Roberta (com ajuste de parâmetros)'
reg_log_roberta_pca = 'Regressão Logística com entrada os embeddings do Roberta (com PCA e Otimização via GridSearch)'

reg_log_bertimbau = 'Regressão Logística com entrada os embeddings do BERTimbau'
reg_log_bertimbau_ajuste = 'Regressão Logística com entrada os embeddings do BERTimbau (com ajuste de parâmetros)'
reg_log_bertimbau_pca = 'Regressão Logística com entrada os embeddings do BERTimbau (com PCA e Otimização via GridSearch)'

reg_log_minilm = 'Regressão Logística com entrada os embeddings do MiniLM'
reg_log_minilm_ajuste = 'Regressão Logística com entrada os embeddings do MiniLM (com ajuste de parâmetros)'
reg_log_minilm_pca = 'Regressão Logística com entrada os embeddings do MiniLM (com PCA e Otimização via GridSearch)'

reg_log_qwen3_06b = 'Regressão Logística com entrada os embeddings do Qwen3 0.6B'
reg_log_qwen3_06b_ajuste = 'Regressão Logística com entrada os embeddings do Qwen3 0.6B (com ajuste de parâmetros)'
reg_log_qwen3_06b_pca = 'Regressão Logística com entrada os embeddings do Qwen3 0.6B (com PCA e Otimização via GridSearch)'

reg_log_qwen3_4b = 'Regressão Logística com entrada os embeddings do Qwen3 4B'
reg_log_qwen3_4b_ajuste = 'Regressão Logística com entrada os embeddings do Qwen3 4B (com ajuste de parâmetros)'
reg_log_qwen3_4b_pca = 'Regressão Logística com entrada os embeddings do Qwen3 4B (com PCA e Otimização via GridSearch)'

reg_log_qwen3_8b = 'Regressão Logística com entrada os embeddings do Qwen3 8B'
reg_log_qwen3_8b_ajuste = 'Regressão Logística com entrada os embeddings do Qwen3 8B (com ajuste de parâmetros)'
reg_log_qwen3_8b_pca = 'Regressão Logística com entrada os embeddings do Qwen3 8B (com PCA e Otimização via GridSearch)'

perspective = 'Perspective'
detoxify = 'Detoxify'

# Realizando o teste

In [16]:
df =
modelo_1 =
modelo_2 =
coluna_para_teste = "macro_f1"

resultado = calcular_teste_t_do_csv(df, modelo_1, modelo_2, coluna_para_teste)

print("-" * 50)
print(f"Teste Estatístico na métrica: {coluna_para_teste}")
print("-" * 50)

if isinstance(resultado, dict):
    for k, v in resultado.items():
        if isinstance(v, float):
            print(f"{k}: {v:.5f}")
        else:
            print(f"{k}: {v}")
else:
    print(resultado)

SyntaxError: invalid syntax (ipython-input-517700634.py, line 1)

# Gráficos

In [17]:
import pandas as pd
import re

# Função para extrair o valor numérico
def extrair_valor_absoluto(texto):
    try:
        # Pega o primeiro grupo de números (float) encontrado na string
        # Ex: "0.736 (0.696, 0.776)" -> pega 0.736
        match = re.search(r"(\d+\.?\d*)", str(texto))
        if match:
            return float(match.group(1))
    except:
        pass
    return -1.0 # Retorna -1 se der erro para ficar no final da lista

dfs_dict = {
    "Misoginia Videos": df_misoginia_videos,
    "Toxic Videos": df_toxic_videos,
    "Misoginia Comentarios": df_misoginia_comentarios,
    "Toxic Comentarios": df_toxic_comentarios
}

for nome, df in dfs_dict.items():
    if 'macro_f1' in df.columns:
        # 1. Cria coluna temporária com o valor float
        df['temp_sort_val'] = df['macro_f1'].apply(extrair_valor_absoluto)

        # 2. Ordena (Ascending=False para decrescente/maior para o menor)
        df.sort_values(by='temp_sort_val', ascending=False, inplace=True)

        # 3. Remove a coluna temporária
        df.drop(columns=['temp_sort_val'], inplace=True)

        # 4. Reseta o index para ficar organizado
        df.reset_index(drop=True, inplace=True)

        print(f"✅ {nome} ordenado com sucesso!")

        display(df[['model/test', 'macro_f1']])
        print("-" * 40)

✅ Misoginia Videos ordenado com sucesso!


,model/test,macro_f1
0,RoBERTa - - desbalanceado,"0.736 (0.696, 0.776)"
1,BerTimbau,"0.722 (0.700, 0.744)"
2,Regressão Logística com entrada os embeddings ...,"0.719 (0.691, 0.746)"
3,Regressão Logística com entrada os embeddings ...,"0.716 (0.681, 0.751)"
4,RoBERTa - - balanceado 60/40,"0.710 (0.645, 0.774)"
...,...,...
58,LLama Few-Shot Embeddings MiniLM (Teste 2),"0.422 (0.400, 0.443)"
59,LLama Few-Shot Embeddings MiniLM (Teste 1),"0.417 (0.380, 0.453)"
60,LLama Few-Shot (Teste 2),"0.411 (0.389, 0.433)"
61,LLama Few-Shot (Teste 1),"0.403 (0.380, 0.427)"


----------------------------------------
✅ Toxic Videos ordenado com sucesso!


,model/test,macro_f1
0,BerTimbau,"0.713 (0.680, 0.747)"
1,RoBERTa - - desbalanceado,"0.712 (0.686, 0.738)"
2,Regressão Logística com entrada os embeddings ...,"0.697 (0.662, 0.732)"
3,Regressão Logística com entrada os embeddings ...,"0.693 (0.626, 0.759)"
4,RoBERTa - - balanceado 60/40,"0.690 (0.650, 0.730)"
...,...,...
58,LLama Few-Shot (Teste 2),"0.353 (0.332, 0.375)"
59,LLama Few-Shot (Teste 1),"0.351 (0.330, 0.373)"
60,Regressão Logística com entrada os embeddings ...,"0.351 (0.097, 0.606)"
61,Regressão Logística com entrada os embeddings ...,"0.338 (0.141, 0.535)"


----------------------------------------
✅ Misoginia Comentarios ordenado com sucesso!


,model/test,macro_f1
0,BerTimbau,"0.800 (0.784, 0.815)"
1,Regressão Logística com entrada os embeddings ...,"0.771 (0.749, 0.794)"
2,RoBERTa - - desbalanceado,"0.770 (0.746, 0.795)"
3,RoBERTa - - balanceado 60/40,"0.766 (0.741, 0.790)"
4,Regressão Logística com entrada os embeddings ...,"0.764 (0.733, 0.796)"
...,...,...
58,Regressão Logística com entrada os embeddings ...,"0.553 (0.301, 0.806)"
59,Regressão Logística com entrada os embeddings ...,"0.491 (0.443, 0.540)"
60,TabPFN -,"0.4249 (0.4226, 0.4324)"
61,Regressão Logística com entrada os embeddings ...,"0.400 (0.228, 0.573)"


----------------------------------------
✅ Toxic Comentarios ordenado com sucesso!


,model/test,macro_f1
0,BerTimbau,"0.754 (0.722, 0.787)"
1,Regressão Logística com entrada os embeddings ...,"0.746 (0.721, 0.771)"
2,Perspective,"0.746 (0.718, 0.773)"
3,Regressão Logística com entrada os embeddings ...,"0.744 (0.713, 0.776)"
4,Regressão Logística com entrada os embeddings ...,"0.739 (0.717, 0.762)"
...,...,...
58,LLama Zero-Shot (Teste 2) ingles sem conceito,"0.587 (0.572, 0.602)"
59,Regressão Logística com entrada os embeddings ...,"0.496 (0.447, 0.545)"
60,Regressão Logística com entrada os embeddings ...,"0.494 (0.449, 0.539)"
61,Regressão Logística com entrada os embeddings ...,"0.420 (0.323, 0.517)"


----------------------------------------


Ordena todos os testes com base no F1-Score para a Classe 1

In [18]:
for nome, df in dfs_dict.items():
    if 'class1f1' in df.columns:
        # 1. Cria coluna temporária com o valor float
        df['temp_sort_val'] = df['class1f1'].apply(extrair_valor_absoluto)

        # 2. Ordena (Ascending=False para decrescente/maior para o menor)
        df.sort_values(by='temp_sort_val', ascending=False, inplace=True)

        # 3. Remove a coluna temporária
        df.drop(columns=['temp_sort_val'], inplace=True)

        # 4. Reseta o index para ficar organizado
        df.reset_index(drop=True, inplace=True)

        print(f"✅ {nome} ordenado com sucesso!")

        display(df[['model/test', 'class1f1']])
        print("-" * 40)

✅ Misoginia Videos ordenado com sucesso!


,model/test,class1f1
0,Regressão Logística com entrada os embeddings ...,"0.581 (0.543, 0.618)"
1,RoBERTa - - balanceado 60/40,"0.581 (0.518, 0.643)"
2,RoBERTa - - desbalanceado,"0.578 (0.513, 0.643)"
3,Regressão Logística com entrada os embeddings ...,"0.567 (0.512, 0.623)"
4,BerTimbau,"0.553 (0.518, 0.588)"
...,...,...
58,Regressão Logística com entrada os embeddings ...,"0.325 (0.211, 0.439)"
59,Regressão Logística com entrada os embeddings ...,"0.252 (0.000, 0.520)"
60,Detoxify,"0.177 (0.096, 0.258)"
61,Regressão Logística com entrada os embeddings ...,"0.167 (0.000, 0.370)"


----------------------------------------
✅ Toxic Videos ordenado com sucesso!


,model/test,class1f1
0,Regressao Logistica TF-IDF (sem stop words + s...,"0.639 (0.604, 0.674)"
1,Regressao Logistica TF-IDF (sem stop words),"0.623 (0.606, 0.641)"
2,Regressao Logistica TF-IDF (stop words padrão ...,"0.601 (0.543, 0.659)"
3,Regressao Logistica TF-IDF (stop words padrão),"0.589 (0.557, 0.621)"
4,Regressao Logistica TF-IDF (stop words customi...,"0.581 (0.548, 0.615)"
...,...,...
58,Regressão Logística com entrada os embeddings ...,"0.330 (0.167, 0.494)"
59,Regressão Logística com entrada os embeddings ...,"0.233 (0.076, 0.390)"
60,Detoxify,"0.216 (0.140, 0.292)"
61,Regressão Logística com entrada os embeddings ...,"0.211 (0.000, 0.424)"


----------------------------------------
✅ Misoginia Comentarios ordenado com sucesso!


,model/test,class1f1
0,BerTimbau,"0.707 (0.681, 0.732)"
1,Regressão Logística com entrada os embeddings ...,"0.677 (0.645, 0.710)"
2,RoBERTa - - balanceado 60/40,"0.673 (0.645, 0.701)"
3,RoBERTa - - balanceado 50/50,0.673 (0.662 - 0.684)
4,Regressão Logística com entrada os embeddings ...,"0.668 (0.614, 0.723)"
...,...,...
58,Regressão Logística com entrada os embeddings ...,"0.372 (0.316, 0.428)"
59,Regressão Logística com entrada os embeddings ...,"0.315 (0.000, 0.730)"
60,Regressão Logística com entrada os embeddings ...,"0.268 (0.046, 0.490)"
61,Regressão Logística com entrada os embeddings ...,"0.167 (0.000, 0.450)"


----------------------------------------
✅ Toxic Comentarios ordenado com sucesso!


,model/test,class1f1
0,LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 4),"0.723 (0.694, 0.752)"
1,LLama Few-Shot Embeddings Qwen3 - 0.6B (Teste 3),"0.719 (0.698, 0.740)"
2,LLama Few-Shot Embeddings Qwen3 - 8B (Teste 1),"0.718 (0.700, 0.735)"
3,LLama Few-Shot Embeddings Qwen3 - 8B (Teste 3),"0.717 (0.700, 0.734)"
4,LLama Few-Shot (Teste 4),"0.716 (0.707, 0.725)"
...,...,...
58,Regressao Logistica TF-IDF (stop words customi...,"0.541 (0.493, 0.589)"
59,Regressão Logística com entrada os embeddings ...,"0.519 (0.464, 0.574)"
60,Regressão Logística com entrada os embeddings ...,"0.479 (0.376, 0.582)"
61,Regressão Logística com entrada os embeddings ...,"0.340 (0.057, 0.624)"


----------------------------------------


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import textwrap # <--- BIBLIOTECA NOVA NECESSÁRIA PARA QUEBRAR O TEXTO

# ==============================================================================
# 1. FUNÇÕES AUXILIARES
# ==============================================================================
# (Função limpar_nomes removida conforme solicitado)

def extrair_metricas(row, col_metrica='macro_f1'):
    """
    Transforma string '0.594 (0.581, 0.607)' em:
    media, erro_para_baixo, erro_para_cima
    """
    val = row[col_metrica]
    try:
        # Busca padrão numérico
        match = re.search(r"(\d+\.?\d*)\s*\(\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*\)", str(val))
        if match:
            mean = float(match.group(1))
            lower = float(match.group(2))
            upper = float(match.group(3))

            # O matplotlib pede o tamanho do erro (distância da média), não o valor absoluto
            err_low = mean - lower
            err_high = upper - mean

            return mean, err_low, err_high
    except:
        pass
    return 0.0, 0.0, 0.0

# ==============================================================================
# 2. PREPARAÇÃO DOS DADOS
# ==============================================================================

# Dicionário com seus DataFrames (assumindo que já estão carregados)
dfs_originais = {
    "Misoginia Vídeos": df_misoginia_videos,
    "Toxicidade Vídeos": df_toxic_videos,
    "Misoginia Comentários": df_misoginia_comentarios,
    "Toxicidade Comentários": df_toxic_comentarios
}

dfs_plot = {}

# Mantive sua lógica de escala fixa entre 0 e 1
escala_min = 0.6
escala_max = 0.9

for nome_df, df in dfs_originais.items():
    d = df.copy()

    # 2. Extração das Métricas
    metrics = d.apply(extrair_metricas, axis=1).tolist()
    d['mean'] = [m[0] for m in metrics]
    d['err_low'] = [m[1] for m in metrics]
    d['err_high'] = [m[2] for m in metrics]

    # 3. Filtrar Top 5 (Ordena pelo maior valor médio)
    d_top5 = d.sort_values(by='mean', ascending=False).head(5)

    # Guarda para plotagem
    dfs_plot[nome_df] = d_top5

# Cria a figura que vai conter os 4 gráficos.
# Tamanho aproximado: 2x a largura (6->12) e 2x a altura (3->6) do individual + margem
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(13, 7))

# Transforma a matriz 2x2 de eixos em uma lista plana para facilitar o loop
axes = axes.flatten()

# Itera simultaneamente sobre os eixos (ax) e os dados (dfs_plot)
for ax, (nome_df, d) in zip(axes, dfs_plot.items()):

    # Ordena Ascendente para o plot
    d_plot = d.sort_values(by='mean', ascending=True)

    y_pos = np.arange(len(d_plot))
    valores = d_plot['mean']
    erros = [d_plot['err_low'], d_plot['err_high']]
    nomes_modelos_brutos = d_plot['model/test']

    # Quebra de linha automática (Mantendo sua config)
    nomes_modelos_embrulhados = [textwrap.fill(nome, width=40) for nome in nomes_modelos_brutos]

    # PLOT PRINCIPAL: Errorbar (Mantendo EXATAMENTE suas cores e tamanhos)
    ax.errorbar(valores, y_pos, xerr=erros, fmt='o', color='#4c72b0',
                ecolor='black', capsize=5, elinewidth=1.5, markeredgewidth=1.5, markersize=9)

    # Configuração dos Eixos
    ax.set_yticks(y_pos)

    # Fonte menor e usando os nomes quebrados
    ax.set_yticklabels(nomes_modelos_embrulhados, fontsize=9)

    # Títulos e Escala
    ax.set_title(nome_df, fontsize=9, fontweight='bold', pad=10)
    ax.set_xlim(escala_min, escala_max)
    ax.set_xlabel("Macro F1 Score", fontsize=8, labelpad=8)

    # Remove Grid e Bordas desnecessárias (Visual limpo)
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.8)

# Ajusta o espaçamento automaticamente para nada ficar sobreposto
plt.tight_layout()
plt.show()